In [2]:
#Step 1: Install Necessary Packages
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import config

In [3]:
#Load API KEY
API_KEY = config.api_key
print("Key Loaded Succesfully")

Key Loaded Succesfully


In [4]:
#Creating Parameters, and informing NASDAQ how many rows we wish to pull
api_url = 'https://data.nasdaq.com/api/v3/datatables/MER/F1.json'

parameters = {
    'api_key': API_KEY,
    'qopts.per_page': 2
}

#Storing data pulled into a variable
json_data = requests.get(api_url, params= parameters).json()

print(json_data)

{'datatable': {'data': [[2438, 1868192544, -1802, 10.481948, '2011-06-30', 'Q2', 'U', 'EUR', 'True', 'Deutsche Bank AG', 'Deutsche Bank AG', 'Active', 'DEU', 'Europe', 1159508, '5.1.1', 'DB', 'NYS', 'Taunusanlage 12', None, None, None, 'Frankfurt am Main', None, 'DEU', '60325', '(49) 69 910 00', '(49) 69 910 34 225', 'www.db.com', '2024-12-31', 'Accrued Expenses Turnover', 'Derived'], [2438, 1868216112, -1802, 8.161754, '2011-09-30', 'Q3', 'U', 'EUR', 'True', 'Deutsche Bank AG', 'Deutsche Bank AG', 'Active', 'DEU', 'Europe', 1159508, '5.1.1', 'DB', 'NYS', 'Taunusanlage 12', None, None, None, 'Frankfurt am Main', None, 'DEU', '60325', '(49) 69 910 00', '(49) 69 910 34 225', 'www.db.com', '2024-12-31', 'Accrued Expenses Turnover', 'Derived']], 'columns': [{'name': 'compnumber', 'type': 'Integer'}, {'name': 'reportid', 'type': 'Integer'}, {'name': 'mapcode', 'type': 'Integer'}, {'name': 'amount', 'type': 'BigDecimal(36,14)'}, {'name': 'reportdate', 'type': 'Date'}, {'name': 'reporttype', 

In [ ]:
data = json_data['datatable']['data']